<a id="table_of_content"></a>

# Part 1: Bag-of-Words Exercises

In the following, we will convert our corpus of 10-k documents into bag-of-words, and convert them into document vectors using Term-Frequency-Inverse-Document-Frequency (tf-idf) re-weighting. Afterward, we will compute sentiments and similarity metrics. Since we will be reusing our notebook, so the various sections are linked below:

1. <a href="#bag_of_word">Compute bag-of-words </a>: implement `bag_of_words` that converts tokenized words into a dictionary of word-counts

2. <a href="#sentiment">Sentiments </a>: using wordlists, compute positive and negative sentiments from bag-of-words. Implement `get_sentiment`

For solutions, see [bagofwords_solutions.py](./bagofwords_solutions.py). You can load the functions by simply calling 

`from bagofwords_solutions import *`

# Part 2: Document-Vector Exercises

3. <a href="#compute_idf">Compute idf </a>: computing the inverse document frequency, implement `get_idf`

4. <a href="#compute_tf">Compute tf </a>: computing the term frequency, implement `get_tf`

5. <a href="#doc_vector">Document vector </a>: using the functions `get_idf` and `get_tf`, compute a word_vector for each document in the corpus
6. <a href="#similarity">Similarities </a>: comparing two vectors, and compute cosine and jacard similarity metrics. Implement `get_cos` and `get_jac`

For solutions, see [bagofwords_solutions.py](./bagofwords_solutions.py). You can load the functions by simply calling 

`from bagofwords_solutions import *`


## 0. Initialization

First we read in our 10-k documents

In [1]:
# download some excerpts from 10-K files
from download10k import get_files

CIK = {'ebay': '0001065088', 'apple':'0000320193', 'sears': '0001310067'}
get_files(CIK['ebay'], 'EBAY')
get_files(CIK['apple'], 'AAPL')
get_files(CIK['sears'], 'SHLDQ')


# get a list of all 10-ks in our directory
files=! ls *10k*.txt
print("10-k files: ",files)
files = [open(f,"r").read() for f in files]

downloading 10-Ks item 1A for CIK = 0001065088 ...
skipping EBAY_10k_2017.txt
skipping EBAY_10k_2016.txt
skipping EBAY_10k_2015.txt
skipping EBAY_10k_2014.txt
skipping EBAY_10k_2013.txt
downloading 10-Ks item 1A for CIK = 0000320193 ...
skipping AAPL_10k_2017.txt
skipping AAPL_10k_2016.txt
skipping AAPL_10k_2015.txt
skipping AAPL_10k_2014.txt
skipping AAPL_10k_2013.txt
downloading 10-Ks item 1A for CIK = 0001310067 ...
skipping SHLDQ_10k_2017.txt
skipping SHLDQ_10k_2016.txt
skipping SHLDQ_10k_2015.txt
skipping SHLDQ_10k_2014.txt
skipping SHLDQ_10k_2013.txt
10-k files:  ['AAPL_10k_2013.txt', 'AAPL_10k_2014.txt', 'AAPL_10k_2015.txt', 'AAPL_10k_2016.txt', 'AAPL_10k_2017.txt', 'EBAY_10k_2013.txt', 'EBAY_10k_2014.txt', 'EBAY_10k_2015.txt', 'EBAY_10k_2016.txt', 'EBAY_10k_2017.txt', 'SHLDQ_10k_2013.txt', 'SHLDQ_10k_2014.txt', 'SHLDQ_10k_2015.txt', 'SHLDQ_10k_2016.txt', 'SHLDQ_10k_2017.txt']


here we define useful functions to tokenize the texts into words, remove stop-words, and lemmatize and stem our words

In [2]:
import numpy as np

# for nice number printing
np.set_printoptions(precision=3, suppress=True)

# tokenize and clean the text
import nltk
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from collections import Counter
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
# tokenize anything that is not a number and not a symbol
word_tokenizer = RegexpTokenizer(r'[^\d\W]+')

nltk.download('stopwords')
nltk.download('wordnet')


sno = SnowballStemmer('english')
wnl = WordNetLemmatizer()

# get our list of stop_words
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 
# add some extra stopwords
stop_words |= {"may", "business", "company", "could", "service", "result", "product", 
               "operation", "include", "law", "tax", "change", "financial", "require",
               "cost", "market", "also", "user", "plan", "actual", "cash", "other",
               "thereto", "thereof", "therefore"}

# useful function to print a dictionary sorted by value (largest first by default)
def print_sorted(d, ascending=False):
    factor = 1 if ascending else -1
    sorted_list = sorted(d.items(), key=lambda v: factor*v[1])
    for i, v in sorted_list:
        print("{}: {:.3f}".format(i, v))

# convert text into bag-of-words
def clean_text(txt):
    lemm_txt = [ wnl.lemmatize(wnl.lemmatize(w.lower(),'n'),'v') \
                for w in word_tokenizer.tokenize(txt) if \
                w.isalpha() and w not in stop_words ]
    return [ sno.stem(w) for w in lemm_txt if w not in stop_words and len(w) > 2 ]

corpus = [clean_text(f) for f in files]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<a id="bag_of_words"></a>

## 1. Bag-of-Words

Implement a function that converts a list of tokenized words into bag-of-words, i.e. a dictionary that outputs the frequency count of a word

** python already provide the `collections.Counter` class to perform this, but I encourage you to implement your own function as an exercise

<a href="#table_of_content">back to top</a>

In [5]:
from collections import defaultdict
from collections import Counter

def bag_of_words(words):
    # TO DO
    count_dict = defaultdict(int)
    for word in words:
        count_dict[word]+=1
    return count_dict


<a id="sentiment"></a>

## 2. Sentiments
Count the fraction of words that appear in a wordlist, returning a sentiment score between 0 and 1:

$$
\textrm{score} = \frac{\textrm{Number of words in document matching wordlist}}{\textrm{Number of words in document}}
$$

Implement the score in a function `get_sentiment(words, wordlist)`, where words is a list of words. Feel free to use the previous `bag_of_words` function. 
(*for extra challenge, try to code the function in one-line*)

Here, I've included a positive and negative wordlist that I constructed by hand. Due to copyright issues, we are not able to provide other commonly used wordlists. I encourage you to try out different wordlists on your own.

<a href="#table_of_content">back to top</a>

In [6]:
# load wordlist first
import pickle

with open('positive_words.pickle', 'rb') as f:
    positive_words = pickle.load(f)
    # also need to stem and lemmatize the text
    positive_words = set(clean_text(" ".join(positive_words)))
    
with open('negative_words.pickle', 'rb') as f:
    negative_words = pickle.load(f)
    negative_words = set(clean_text(" ".join(negative_words)))
    
# check out the list
print("positive words: ", positive_words)
print("negative words: ", negative_words)

positive words:  {'bright', 'solut', 'celebr', 'miracul', 'best', 'except', 'revolutionari', 'stabil', 'resolv', 'fortun', 'perfect', 'beauti', 'congratul', 'creativ', 'eleg', 'visionari', 'clear', 'astonish', 'popular', 'flourish', 'applaus', 'trendi', 'famous', 'victori', 'nurtur', 'glamor', 'higher', 'innov', 'abund', 'wholesom', 'happi', 'command', 'prosper', 'achiev', 'excel', 'approv', 'satisfi', 'surpass', 'paradis', 'special', 'heal', 'improv', 'recov', 'love', 'aptitud', 'enrich', 'persever', 'boost', 'clean', 'effici', 'boon', 'stupend', 'outperform', 'dazzl', 'booster', 'progress', 'bolster', 'fresh', 'master', 'resourc', 'groundbreak', 'spring', 'delight', 'effect', 'dynam', 'harmoni', 'motiv', 'restor', 'honor', 'plenti', 'winner', 'reward', 'divin', 'benign', 'enhanc', 'optim', 'respect', 'monument', 'advanc', 'buoy', 'elev', 'revolution', 'attract', 'forefront', 'magic', 'independ', 'jubil', 'wealth', 'awesom', 'prais', 'compliment', 'fabul', 'endors', 'optimist', 'incre

In [7]:
def get_sentiment(txt, wordlist):
    # TO DO
    matching_words = [w for w in txt if w in wordlist]
    return len(matching_words)/len(txt)

In [8]:
# test your function
positive_sentiments = np.array([ get_sentiment(c, positive_words) for c in corpus ])
print(positive_sentiments)

negative_sentiments = np.array([ get_sentiment(c, negative_words) for c in corpus ])
print(negative_sentiments)

[ 0.034  0.033  0.032  0.032  0.031  0.034  0.033  0.034  0.031  0.032
  0.037  0.033  0.037  0.039  0.04 ]
[ 0.054  0.053  0.052  0.052  0.053  0.051  0.05   0.054  0.061  0.061
  0.06   0.059  0.058  0.054  0.059]


**before continuing part 2, go through the lesson material first!**

<a id="compute_idf"></a>

# Part 2 Document Vector Exercises

## 3. Computer idf
Given a corpus, or a list of bag-of-words, we want to compute for each word $w$, the inverse-document-frequency, or ${\rm idf}(w)$. This can be done in a few steps:

1. Gather a set of all the words in all the bag-of-words (python set comes in handy, and the union operator `|` is useful here)


2. Loop over each word $w$, and compute ${\rm df}_w$, the number of documents where this word appears at least once. A dictionary is useful for keeping track of ${\rm df}_w$


3. After computing ${\rm df}_w$, we can compute ${\rm idf}(w)$. There are usually two possibilities, the simplest one is 
$${\rm idf}(w)=\frac{N}{{\rm df}_w}$$
Where $N$ is the total number of documents in the corpus and $df_w$ the number of documents that contain the word $w$. Frequently, a logarithm term is added as well
$${\rm idf}(w)=\log\frac{N}{{\rm df}_w}$$
One issue with using the logarithm is that when ${\rm df}_w = N$, ${\rm idf}(w)=0$, indicating that words common to all documents would be ignored. If we don't want this behavior, we can define ${\rm idf}(w)=\log\left(1+N/{\rm df}_w\right)$ or ${\rm idf}(w)=1+\log\left(N/{\rm df}_w\right)$ instead. For us, we'll not use the extra +1 for ${\rm idf}$.

In the following, define a function called `get_idf(corpus, include_log=True)` that computes ${\rm idf}(w)$ for all the words in a corpus, where `corpus` for us is a processed list of bag-of-words (stemmed and lemmatized). The optional parameter `include_log` includes the logarithm in the computation.

<a href="#table_of_content">back to top</a>

In [11]:
# compute idf
def get_idf(corpus, include_log=True):
    # TO DO
    
    all_words = set()
    for bag_of_words in corpus:
        all_words |= set(bag_of_words)
    
    word_df_dict = defaultdict(int)
    
    for word in all_words:
        for doc in corpus:
            if word in doc:
                word_df_dict[word]+=1
    
    word_idf_dict = defaultdict(float)
    for key, value in word_df_dict.items():
        word_idf_dict[key] = np.log(len(corpus)/value) if include_log else len(corpus)/value
    
    return word_idf_dict

You should expect to see many idf values = 0! This is by design, because we have ${\rm idf}(w)=\log N_d/{\rm df}_w$ and $N_d/{\rm df}_w=1$ for the most common words!

In [12]:
# test your code
idf=get_idf(corpus)
print_sorted(idf, ascending=True)

howev: 0.000
identifi: 0.000
reli: 0.000
outsourc: 0.000
defend: 0.000
due: 0.000
brand: 0.000
adequ: 0.000
accord: 0.000
condit: 0.000
sell: 0.000
extens: 0.000
attract: 0.000
secur: 0.000
depend: 0.000
pay: 0.000
determin: 0.000
commerci: 0.000
suffer: 0.000
event: 0.000
reason: 0.000
order: 0.000
fire: 0.000
oper: 0.000
loss: 0.000
ship: 0.000
manufactur: 0.000
reput: 0.000
limit: 0.000
usag: 0.000
asset: 0.000
facil: 0.000
rate: 0.000
local: 0.000
procedur: 0.000
retail: 0.000
subject: 0.000
amount: 0.000
direct: 0.000
manag: 0.000
act: 0.000
devot: 0.000
signific: 0.000
part: 0.000
exist: 0.000
expens: 0.000
interest: 0.000
reduc: 0.000
contract: 0.000
increas: 0.000
therebi: 0.000
thus: 0.000
action: 0.000
past: 0.000
oblig: 0.000
outstand: 0.000
expect: 0.000
incur: 0.000
certain: 0.000
base: 0.000
interrupt: 0.000
otherwis: 0.000
payment: 0.000
suppli: 0.000
system: 0.000
face: 0.000
level: 0.000
comput: 0.000
challeng: 0.000
deliveri: 0.000
person: 0.000
high: 0.000
alleg: 0.0

<a id="compute_tf"></a>

## 4. Compute tf

Below we will compute ${\rm tf}(w,d)$, or the term frequency for a given word $w$, in a given document $d$. Since our ultimate goal is to compute a document vector, we'd like to keep a few things in mind

1. Store the ${\rm tf}(w, d)$ for each word in a document as a dictionary
2. Even when a word doesn't appear in the document $d$, we still want to keep a $0$ entry in the dictionary. This is important when we convert the dictionary to a vector, where zero entries are important


There are multiple definitions for ${\rm tf}(w,d)$, the simplest one is

$$
{\rm tf}(w,d)=\frac{f_{w,d}}{a_d}
$$

where $f_{w,d}$ is the number of occurence of the word $w$ in the document $d$, and $a$ the average occurence of all the words in that document for normalization. Just like ${\rm idf}(w)$, a logarithm can be added

$$
{\rm tf}(w,d)=\begin{cases}
\frac{1+\log f_{w,d}}{1+\log a_d} & f_{w,d} > 0 \\
0 & f_{w,d} = 0 \\
\end{cases}
$$

Implement the function `get_df(txt, include_log=True)` that computes ${\rm tf}(w,d)$ for each word in the document (returns a defaultdict(int), so that when supplying words not in the document the dictionary will yield zero instead of an error). Also include the optional parameter `include_log` that enables the additional logarithm term in the computation. I suggest also adding a function called `_tf` that computes the function above. 

<a href="#table_of_content">back to top</a>

In [17]:
import numpy as np
from math import *

def _tf(freq, avg, include_log=True):
    # TO DO
    if include_log:
        return (1+np.log(freq))/(1+np.log(avg)) if freq > 0 else 0
    else:
        return freq/avg

def get_tf(txt, include_log=True):
    # TO DO
    bow = bag_of_words(txt)
    avg = np.mean(list(bow.values()))
    tf = defaultdict(int)
    tf = {w:_tf(f, avg, include_log) for w, f in bow.items()}
    return tf
    
    

In [18]:
tfs = [ get_tf(c) for c in corpus ]
print_sorted(tfs[0])

signific: 1.920
compon: 1.920
risk: 1.855
develop: 1.855
affect: 1.841
price: 1.841
subject: 1.841
new: 1.811
parti: 1.795
continu: 1.761
custom: 1.761
third: 1.761
advers: 1.743
increas: 1.705
oper: 1.705
system: 1.684
technolog: 1.684
intern: 1.684
foreign: 1.684
sale: 1.663
softwar: 1.663
applic: 1.663
manag: 1.663
relat: 1.640
substanti: 1.640
limit: 1.640
materi: 1.616
valu: 1.616
demand: 1.616
addit: 1.616
manufactur: 1.616
experi: 1.616
condit: 1.590
credit: 1.590
effect: 1.590
partner: 1.590
invest: 1.590
competit: 1.590
compet: 1.590
econom: 1.563
depend: 1.563
mani: 1.563
currenc: 1.563
supplier: 1.563
secur: 1.563
margin: 1.563
time: 1.563
devic: 1.563
avail: 1.563
retail: 1.563
content: 1.534
distribut: 1.534
provid: 1.534
assur: 1.534
inform: 1.534
store: 1.534
regul: 1.534
industri: 1.503
failur: 1.503
obtain: 1.503
futur: 1.503
purchas: 1.469
loss: 1.469
competitor: 1.469
intellectu: 1.469
properti: 1.469
certain: 1.469
disrupt: 1.469
jurisdict: 1.469
trade: 1.469
resel:

<a id="doc_vector"></a>

## 5. Document Vector
Combine the implementation for ${\rm tf}(w,d)$ and ${\rm idf}(w)$ to compute a word-vector for each document in a corpus. Don't forget the zero-padding that is needed when a word appears in some document but not others. 

Implmenet the function `get_vectors(tf,idf)`, the input is the output computed by `get_tf` and `get_idf`

(*optional challenge: implement in one line!*)

<a href="#table_of_content">back to top</a>

In [45]:
def get_vector(tf, idf):
    vector = []
    for w in idf:
        if w not in tf:
            np.insert(vector, 0)
        else:
            np.insert(vector, (tf[w]*idf[w]))
    return vector
    

In [46]:
# test your code
doc_vectors = [ get_vector(tf, idf) for tf in tfs ]

for v in doc_vectors:
    print(v)

TypeError: insert() missing 1 required positional argument: 'values'

<a id ="similarity"></a>

## 6. Similarity

Given two word-vectors $\vec u$ (or $u_i$) and $\vec v$ (or $v_i$), corresponding to two documents, we want to compute different similarity metrics. 

1. Cosine similarity, defined by 
$$
{\rm Sim}_{\cos} = \frac{\vec u \cdot \vec v}{|\vec u| |\vec v|}
$$

2. Jaccard similarity, defined by
$$
{\rm Sim}_{\rm Jaccard} = \frac{\sum_i \min\{u_i, v_i\}}{\sum_i \max\{u_i, v_i\}}
$$

Implement the function similarity as `sim_cis(u,v)` and `sim_jac(u,v)`. Utilize the numpy functions `numpy.linalg.norm` to compute norm of a vector and `np.dot` for computing dot-products. `np.minimum` and `np.maximum` are also useful vectorized pair-wise minimum and maximum functions.

(*optional challenge: implement both functions in one line!*)

<a href="#table_of_content">back to top</a>

In [39]:
from numpy.linalg import norm

def sim_cos(u,v):
    # TO DO
    return np.dot(u, v)/(norm(u)*norm(v))

def sim_jac(u,v):
    # TO DO
    return np.sum(np.minimum(u, v))/np.sum(np.maximum(u, v))

In [40]:
# test your co
# compute all the pairwise similarity metrics
size = len(doc_vectors)
matrix_cos = np.zeros((size,size))
matrix_jac = np.zeros((size,size))

for i in range(size):
    for j in range(size):
        u = doc_vectors[i]
        v = doc_vectors[j]
        matrix_cos[i][j] = sim_cos(u,v)
        matrix_jac[i][j] = sim_jac(u,v)
        
print("Cosine Similarity:")
print(matrix_cos)

print()
print("Jaccard Similarity:")
print(matrix_jac)

NameError: name 'doc_vectors' is not defined

### Good Job! You've finished all the exercises!

Here is an optional bonus challenge. We often need to debug other people's code to figure out what's wrong. It's particularly difficult when the code doesn't give errors but computes the wrong quantity. Can you describe why the following implementations for some of the exercises above may be wrong? Highlight the words at the bottom to reveal the solutions!

In [47]:
def get_idf_wrong(corpus, include_log=True):
    freq = defaultdict(int)
    for c in corpus:
        for w in c:
            freq[w] += 1
        
    N = len(corpus)
    if include_log:
        return { w:log(N/freq[w]) for w in freq }
    else:
        return { w:N/freq[w] for w in freq }


def get_sentiment_wrong(txt, wordlist):
    matching_words = [ w for w in wordlist if w in txt ]
    return len(matching_words)/len(txt)

def get_vectors_wrong(tf, idf):
    return np.array([ tf[w]*idf[w] for w in tf ])

# Solutions

Drag your mouse over the white space below this cell, and you'll see details about the solutions.  Or, if it's easier, just double-click on the white space below this cell, which will reveal the cell with hidden text.  Also, please check out the file [bagofwords_solutions.py](bagofwords_solutions.py).

<font color="white">
Solution

get_idf: the defaultdict freq here computes the total number of occurences in all the document. We only want to count it once when a word appears in a document. This may lead to a document frequency larger than N, leading to a negative idf! 

get_sentiment_wrong: if a word w appears twice in the document, it won't be counted properly!

get_vectors_wrong: tf may not contain all the words in idf. We need zero padding for words that appear in idf but not in tf! 
</font> 